## 1. Import required packages and waterway network under study

In [1]:
import networkx as nx
import pandas as pd
import pickle
G = pickle.load( open('data/network_digital_twin_v0.2.pickle','rb'))

## 2. Define sub-modelling problem
Harbours (where a charging station may be place) are almost never directly passed by ships on ongoing routes.
Hence the following questions where raised:
- Which locations may be considered to place charging stations ?
- How to know when a potential location is passed (because not directly on route) ?

The overall goal is thus to create a initial list with nodes, on ongoing routes that lead to harbours. This list may be changed based
on new insights later on. Using this list is a simplification of the problem, ideally the time and distance to travel in and out of a
harbour should also be considered.

## 3. Approach
First branch point towards a port is taken as the entrance to a port. The port itself thus may still be a few nodes further inside
(see picture below). This seems a reasonable assumption, as a harbour also often has more than one dock (also see picture below).


![Tilburg](Tilburg.PNG)

Note that in the picture below, node 8861748 is thus considered to place a charging station, instead of node 8862372.
In essence this won't make much of a difference, as the inland node is always quite near, and the distance is not very significant.

All links that go towards a harbour, have a specific name tag: 'Vaarwegvak van 0 tot 0 - H'
The first (most inland) node of a port is usually connected to a ongoing route, the others are not connected to such a node.
These nodes, can thus be recognised by this tag in combination of a degree higher than or equal to three.
Hence, all nodes are dropped if they have a degree smaller than 3 (see code below).

### Process that resulted in the current pickled list with harbour nodes:

In [2]:
#Get data from df
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [3]:
#visual inspection of the network showed that all harbour related nodes are tagged: ‘Vaarwegvak van 0 tot 0 - H’
#However, these are many nodes, and each harbour often consists of at least a couple.
df_links.loc[(df_links.Name=='Vaarwegvak van 0 tot 0 - H')]

,source,target,lat_p50,Code,nap_p10,nap_p5,CoupledDepth,Id_navigability,lat_p0,Length,...,GeneralWidth,PushedWidth,WidePushedLength,lat_mean,SeaFairingHeight,GeneralLength,Name,WidePushedWidth,PushedDepth,CoupledWidth
8,8861581,8863189,NaN,_0,NaN,NaN,NaN,25496406.0,NaN,0.371,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
10,8860954,8864050,NaN,VI_C,NaN,NaN,NaN,10916.0,NaN,0.785,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
15,8865802,8862540,NaN,V_A,NaN,NaN,NaN,8090.0,NaN,0.465,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
16,8861605,8863756,NaN,V_A,NaN,NaN,NaN,25487342.0,NaN,0.232,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
22,8864595,8864137,NaN,VI_C,NaN,NaN,NaN,42146.0,NaN,0.104,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14479,Berth20,Berth22,NaN,VI_C,0.968794,0.900677,NaN,45971.0,NaN,0.349,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
14480,Berth21,8860919,NaN,VI_C,1.342185,1.263797,NaN,45971.0,NaN,0.349,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
14481,Berth21,Berth22,NaN,VI_C,1.347982,1.347982,NaN,45971.0,NaN,0.349,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
14487,Berth39,Berth40,NaN,V_A,-6.139389,-6.148712,NaN,44403.0,NaN,0.595,...,16.0,NaN,NaN,NaN,NaN,140.0,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN


In [4]:
#There are Berths inside the df that are tagged
df_links.loc[(df_links.target.str.contains('Berth'))].dropna(how='all')

,source,target,lat_p50,Code,nap_p10,nap_p5,CoupledDepth,Id_navigability,lat_p0,Length,...,GeneralWidth,PushedWidth,WidePushedLength,lat_mean,SeaFairingHeight,GeneralLength,Name,WidePushedWidth,PushedDepth,CoupledWidth
159,8864666,Berth19,NaN,V_B,2.462781,2.412057,NaN,25343.0,NaN,1.299,...,15.5,NaN,NaN,NaN,NaN,225.0,Vaarwegvak van 10 tot 12 - H,NaN,NaN,NaN
282,8862842,Berth13,NaN,IV,-4.866506,-5.066942,NaN,52393.0,NaN,1.000,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 1 - H,NaN,NaN,NaN
283,8862842,Berth40,NaN,V_A,-5.803653,-5.848389,NaN,44403.0,NaN,0.595,...,16.0,NaN,NaN,NaN,NaN,140.0,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
351,8866538,Berth23,NaN,_0,NaN,NaN,NaN,25496547.0,NaN,2.898,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 2 tot 5 - H,NaN,NaN,NaN
1105,28313551,Berth29,NaN,II,-6.774988,-6.791406,NaN,44774.0,NaN,0.890,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14481,Berth21,Berth22,NaN,VI_C,1.347982,1.347982,NaN,45971.0,NaN,0.349,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
14483,Berth24,Berth25,NaN,V_A,12.645116,12.564657,NaN,52183.0,NaN,11.533,...,15.5,13.5,NaN,NaN,NaN,137.5,Vaarwegvak van 30 tot 41 - H,NaN,3.0,NaN
14484,Berth31,Berth32,NaN,V_B,-5.735271,-5.740968,NaN,36960.0,NaN,2.009,...,17.5,NaN,NaN,NaN,NaN,190.0,Vaarwegvak van 32 tot 34 - H,NaN,NaN,NaN
14485,Berth34,Berth35,NaN,V_B,-6.770000,-6.800000,NaN,36960.0,NaN,7.324,...,17.5,NaN,NaN,NaN,NaN,190.0,Vaarwegvak van 34 tot 41 - H,NaN,NaN,NaN


In [5]:
#However,
df_links.loc[(df_links.Name=='Vaarwegvak van 0 tot 0 - H')& (df_links.target.str.contains('Berth'))].dropna(how='all')
# results in only 15 rows, for 47 Berths, doesn't work too well...
# Sometimes multiple links to final harbour node...
# What if Vaarwegvak van 0 tot 0 - H, with at least degree of 3 of source node?

,source,target,lat_p50,Code,nap_p10,nap_p5,CoupledDepth,Id_navigability,lat_p0,Length,...,GeneralWidth,PushedWidth,WidePushedLength,lat_mean,SeaFairingHeight,GeneralLength,Name,WidePushedWidth,PushedDepth,CoupledWidth
283,8862842,Berth40,NaN,V_A,-5.803653,-5.848389,NaN,44403.0,NaN,0.595,...,16.0,NaN,NaN,NaN,NaN,140.0,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
1105,28313551,Berth29,NaN,II,-6.774988,-6.791406,NaN,44774.0,NaN,0.890,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
1652,8866129,Berth29,NaN,II,-5.818934,-6.237931,NaN,44774.0,NaN,0.890,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
1927,8861452,Berth20,NaN,VI_C,1.317915,1.293197,NaN,45971.0,NaN,0.349,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
1994,8863471,Berth28,NaN,VI_B,-9.653448,-11.447551,NaN,25495526.0,NaN,0.855,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
3277,8867489,Berth8,NaN,VI_B,-11.952581,-12.268987,NaN,49182.0,NaN,0.402,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
3278,8867489,Berth39,NaN,V_A,-13.795342,-13.860395,NaN,44403.0,NaN,0.595,...,16.0,NaN,NaN,NaN,NaN,140.0,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
5281,B7532_A,Berth28,NaN,VI_B,-2.030411,-2.143528,NaN,25495526.0,NaN,0.855,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
14464,22638160,Berth48,NaN,II,-4.775776,-4.816955,NaN,25496287.0,NaN,0.221,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
14465,8862253,Berth4,NaN,V_A,-5.600933,-5.654598,NaN,58223.0,NaN,0.297,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN


In [6]:
#first collect degrees
degrees = G.degree

harbour_links = df_links.loc[(df_links.Name=='Vaarwegvak van 0 tot 0 - H')]
harbour_links.reset_index(inplace = True, drop = True)

for index, source in enumerate(harbour_links.source):
    if degrees[source] < 3:
        harbour_links.drop(index, inplace=True)

harbour_links.reset_index(inplace = True, drop = True)

harbour_links
#looks OK for now! Maybe still filter based on depth or something like that later on

C:\Users\Fabia\AppData\Local\Temp\ipykernel_14096\4039122636.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  harbour_links.drop(index, inplace=True)
C:\Users\Fabia\AppData\Local\Temp\ipykernel_14096\4039122636.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  harbour_links.drop(index, inplace=True)
C:\Users\Fabia\AppData\Local\Temp\ipykernel_14096\4039122636.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  harbour_links.drop(i

,source,target,lat_p50,Code,nap_p10,nap_p5,CoupledDepth,Id_navigability,lat_p0,Length,...,GeneralWidth,PushedWidth,WidePushedLength,lat_mean,SeaFairingHeight,GeneralLength,Name,WidePushedWidth,PushedDepth,CoupledWidth
0,8861581,8863189,NaN,_0,NaN,NaN,NaN,25496406.0,NaN,0.371,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
1,8860954,8864050,NaN,VI_C,NaN,NaN,NaN,10916.0,NaN,0.785,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
2,8865802,8862540,NaN,V_A,NaN,NaN,NaN,8090.0,NaN,0.465,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
3,8864595,8864137,NaN,VI_C,NaN,NaN,NaN,42146.0,NaN,0.104,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
4,8864595,8862678,NaN,VI_C,NaN,NaN,NaN,53505.0,NaN,0.628,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,8868094,B51576_A,NaN,II,NaN,NaN,NaN,6367.0,NaN,0.243,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
811,S46058_A,S46058_A,NaN,V_A,2.676762,2.676762,NaN,25487825.0,NaN,0.708,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
812,S46058_A,S46058_B,NaN,V_A,2.713659,2.713659,NaN,25487825.0,NaN,0.708,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN
813,22638160,Berth48,NaN,II,-4.775776,-4.816955,NaN,25496287.0,NaN,0.221,...,NaN,NaN,NaN,NaN,NaN,NaN,Vaarwegvak van 0 tot 0 - H,NaN,NaN,NaN


In [7]:
print('# harbour links', len(harbour_links))
# number of unique harbour_exits for whole network looks somewhat reasonable

# harbour links 815


In [8]:
#unique sources should be on ongoing routes
harbour_exits = list(harbour_links.source.unique())
print(len(harbour_exits))

649


In [9]:
# pick list to use in program
pickle.dump( harbour_exits, open( "data/harbour_exits.p", "wb" ) )